In [3]:
import numpy as np
import pandas as pd
import pystan
from scipy.stats import norm, multivariate_normal, invwishart, invgamma
import datetime
import sys
import os

from codebase.plot import * 
from codebase.file_utils import save_obj, load_obj

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
log_dir = "log/20190712_002112_test-screen2/"

In [44]:
file = open(log_dir+"model.txt", "r") 
print(file.read())
file.close()

data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  matrix[N,J] yy;
}

parameters {
  vector<lower=0>[J] sigma;
  vector<lower=0>[K] sigma_z;
  vector[J] alpha;
  matrix[2,K] beta_free; // 2 free eleements per factor
  cholesky_factor_corr[K] Phi_corr_chol;
}

transformed parameters{
  cov_matrix[J] Theta;
  matrix[J,K] beta;
  cov_matrix [K] Phi_cov ;
  cov_matrix[J] Marg_cov;

  Theta = diag_matrix(square(sigma));
  Phi_cov = multiply_lower_tri_self_transpose(diag_pre_multiply(sigma_z, Phi_corr_chol));

  for(j in 1:J) {
    for (k in 1:K) beta[j,k] = 0;
  }

  // set ones
  for (k in 1:K) beta[1+3*(k-1), k] = 1;

  // set the free elements
  for (k in 1:K) beta[2+3*(k-1) : 3+3*(k-1), k] = beta_free[1:2,k];

  Marg_cov = beta * Phi_cov * beta'+ Theta;
}

model {
  to_vector(beta_free) ~ normal(0, 1);
  to_vector(alpha) ~ normal(0, 1);
  sigma ~ cauchy(0,1);
  sigma_z ~ cauchy(0,1);
  Phi_corr_chol ~ lkj_corr_cholesky(2);
  for (n in 1:N){
    yy[n, ] ~ multi_normal(alpha, 

In [45]:
data = load_obj("stan_data", log_dir)

In [46]:
ps = load_obj('ps', log_dir)


In [47]:
%%opts Layout [fig_size=200]
plots = []
for j in range(data['J']):
    for k in range(data['K']):
        plots.append(plot_trace(ps['beta'][:,j,k],
             title = 'Posterior distribution for beta(%s,%s)'%(j,k)).\
                     options(fig_inches=8, aspect=3))
layout = hv.Layout(plots).options(show_title = True,
                                  vspace = .3,
                                  absolute_scaling=False,
                                  normalize=False) # use same y-range for all plots?

layout.cols(2)


:Layout
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_2_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_3_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_4_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_2_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_3_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_4_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_2_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_2_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_2_comma_2_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_2_comma_3_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_2_comma_4_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_3_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_3_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_3_comma_2_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_3_comma_3_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_3_comma_4_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_4_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_4_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_4_comma_2_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_4_comma_3_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_4_comma_4_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_5_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_5_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_5_comma_2_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_5_comma_3_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_5_comma_4_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_6_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_6_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_6_comma_2_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_6_comma_3_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_6_comma_4_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_7_comma_0_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_7_comma_1_right_parenthesis  :Curve   [x]   (y)
   .Curve.Posterior_distribution_

In [48]:
# %%output info=True 
plots = []
for j in range(data['K']):
    for k in range(data['K']):
        if j!=k:
            plots.append(plot_trace(ps['Phi_cov'][:,j,k],
            title = 'Posterior distribution for Phi(%s,%s)'%(j,k)).options(fig_inches=10, aspect=3))
layout = hv.Layout(plots).options(show_title = True,
#     tight = True, tight_padding=10,
    vspace = .5,
    absolute_scaling=False, fig_size=100)
layout.cols(2)

:Layout
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_0_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_0_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_0_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_1_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_1_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_1_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_2_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_2_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_2_comma_3_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_2_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_3_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_3_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_3_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_3_comma_4_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_4_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_4_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_4_comma_2_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_Phi_left_parenthesis_4_comma_3_right_parenthesis :Curve   [x]   (y)

In [22]:
mle_est = dict()
mle_est['Sigma'] = np.cov(data['y'], rowvar=False)
mle_est['mu'] = np.mean(data['y'], axis=0)
def compute_D1(yy):   
    return multivariate_normal.logpdf(yy, mean=mle_est['mu'], cov=mle_est['Sigma'])

In [24]:
def compute_D2(yy, mcmc_iter):    
    D2 = multivariate_normal.logpdf(yy,
                               mean= ps['alpha'][mcmc_iter],
                               cov = ps['Marg_cov'][mcmc_iter])
    return D2

def compute_D(mcmc_iter, pred=True):
    if pred == True:
        y_pred=multivariate_normal.rvs(mean= ps['alpha'][mcmc_iter],
                        cov=ps['Marg_cov'][mcmc_iter],
                       size = data['y'].shape[0])
        return compute_D1(y_pred) - compute_D2(y_pred, mcmc_iter)

    else:
        return compute_D1(data['y']) - compute_D2(data['y'], mcmc_iter)
    
    
compute_D(100, False)

array([ 3.84649755e-01,  5.24387135e-01,  7.04612764e-01,  2.58697657e+00,
       -4.97992968e-01,  1.37286499e+00, -7.86442694e-02,  6.79697893e-01,
        2.62806396e-01,  1.08496388e+00,  3.43503770e-01,  3.43548484e-01,
        3.94092553e-01, -2.10010970e-01,  1.34446059e-01,  1.42939746e-01,
       -5.01666532e-01,  2.36796957e-02,  1.63980834e-01,  1.75548840e-01,
       -7.73404513e-02,  1.33936299e-01, -1.04122103e+00, -2.80247980e-02,
        8.71329902e-01,  3.19687778e-01,  3.15366554e-01,  8.69921949e-01,
        7.35551361e-01,  1.45681822e+00,  6.16266549e-01,  1.05777617e+00,
        4.73762063e-01,  1.43146144e-01, -3.87782776e-01,  3.86606636e-01,
        3.14846422e-01,  2.71405600e-01,  7.60629863e-01,  9.97407702e-01,
        4.29507046e-01,  1.41661539e-01,  3.22562799e-01,  1.80862696e-01,
       -1.00234204e-02,  1.24881287e+00,  8.19738953e-01,  6.32704478e-01,
        2.01720152e-01,  1.70768491e-01,  3.53514201e-01,  4.66935491e-01,
        1.44810030e+00,  

In [28]:
np.sum(compute_D(mcmc_iter, pred=True))

-184.8582159852987

In [25]:
mcmc_length = 1000
Ds = np.empty((mcmc_length,2))
for mcmc_iter in range(mcmc_length):
    Ds[mcmc_iter,0] = -2*np.sum(compute_D(mcmc_iter, pred=False))
    Ds[mcmc_iter,1] = -2*np.sum(compute_D(mcmc_iter, pred=True))

In [26]:
Ds[:,0] < Ds[:,1]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,